In [29]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pycbc.filter import sigmasq, sigma
from pycbc.psd import aLIGOZeroDetHighPower
from simple_pe.waveforms import make_waveform, shifted_e

import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [85]:
def chirp_degeneracy_line(input_chirp, other_params, ecc, sample_rate, f_low, psd, approximant):
    
    # Generate waveform at non-eccentric point to use in sigmasq
    # if 'ecc10' in other_params:
    #     assert other_params['ecc10'] == 0
    # else:
    #     other_params['ecc10'] = 0
    other_params['chirp_mass'] = input_chirp
    h = make_waveform(other_params, psd.delta_f, f_low, len(psd), approximant=approximant)

    # Handle array of eccentricities as input
    array = False
    if len(np.shape(ecc)) > 0:
        array = True
    ecc = np.array(ecc).flatten()

    ssfs = np.zeros(len(ecc))
    ssffs = np.zeros(len(ecc))
    sskfs = np.zeros(len(ecc))
    sskffs = np.zeros(len(ecc))
    
    sskkffs = np.zeros(len(ecc))
    
    # Loop over each eccentricity
    for i, e in enumerate(ecc):
        
        # Calculate eccentricities and k values
        s_es = shifted_e(h.sample_frequencies, f_low, e)
        ks_sqrt = np.sqrt(2355*s_es**2/1462)
    
        # Calculate and normalise integrals
        ss = sigmasq(h, psd=psd, low_frequency_cutoff=f_low)
        ssf = sigmasq(h*h.sample_frequencies**(-5/6), psd=psd, low_frequency_cutoff=f_low)
        ssff = sigmasq(h*h.sample_frequencies**(-5/3), psd=psd, low_frequency_cutoff=f_low)
        sskf = -sigmasq(h*ks_sqrt*h.sample_frequencies**(-5/6), psd=psd, low_frequency_cutoff=f_low)
        sskff = -sigmasq(h*ks_sqrt*h.sample_frequencies**(-5/3), psd=psd, low_frequency_cutoff=f_low)

        sskkff = sigmasq(h*ks_sqrt**2*h.sample_frequencies**(-5/3), psd=psd, low_frequency_cutoff=f_low)
        
        # ssfs[i], ssffs[i], sskfs[i], sskffs[i] = np.array([ssf, ssff, sskf, sskff])/ss

        ssfs[i], ssffs[i], sskfs[i], sskffs[i], sskkffs[i] = np.array([ssf, ssff, sskf, sskff, sskkff])/ss

    # Calculate chirp mass
    # delta_m = - (sskffs - ssfs*sskfs)/(ssffs - ssfs**2)
    delta_m_num = sskffs + sskkffs - ssfs*sskfs - sskfs**2
    delta_m_denom = ssffs + sskffs - ssfs**2 - ssfs*sskfs
    delta_m = -delta_m_num/delta_m_denom
    chirp = input_chirp*(1+delta_m)**(3/5)

    # If array not passed then turn back into float
    if not array:
        chirp = chirp[0]
    return chirp    

In [80]:
def find_zero_ecc_chirp(params, sample_rate, f_low, psd, approximant):

    init_guess = params['chirp_mass']
    zero_ecc_params = params.copy()
    ecc = shifted_e(f_low, 10, zero_ecc_params.pop('ecc10'))
    ecc_chirp = zero_ecc_params.pop('chirp_mass')
    min_func = lambda x: abs(chirp_degeneracy_line(x, zero_ecc_params, ecc, sample_rate, f_low, psd, approximant) - ecc_chirp)
    result = minimize(min_func, init_guess, bounds=[(0.5*init_guess, 1.5*init_guess)], method='Nelder-Mead', options={'fatol': 0.01, 'xatol': 0.01})
    zero_ecc_params['ecc10'] = 0
    zero_ecc_params['chirp_mass'] = result['x']
    return zero_ecc_params

The equations of this degeneracy line rely upon knowledge of the chirp mass at zero eccentricity (i.e. circular chirp mass). Calculating an eccentric point based on the circular point is therefore very easy:

In [86]:
psd = aLIGOZeroDetHighPower((32*4096)//2 + 1, 1/32, 10)
chirp = chirp_degeneracy_line(22.16087952281798, {'symmetric_mass_ratio': 2/9, 'chi_eff': 0.5, 'ecc10': 0.4}, 0.4, 4096, 10, psd, 'TEOBResumS-Dali')
print(chirp)

23.380978976709113


Instead we wish to do the reverse, i.e. to calculate the circular point given an eccentric point. Here we turn to `scipy.optimize.minimize` to reverse the function at the cost of increased computation time as a circular waveform must be generated for each function evaluation.

In [8]:
params = {'ecc10': 0.4, 'chirp_mass': 22.16087952281798, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0.5}
start = time.time()
zero_ecc_params = find_zero_ecc_chirp(params, 4096, 10, psd, approximant='TEOBResumS-Dali')
print(zero_ecc_params)
end = time.time()
print(f'Calculated in {end-start:.3f} seconds')

{'symmetric_mass_ratio': Array([0.22222222]), 'chi_eff': Array([0.5]), 'ecc10': 0, 'chirp_mass': array([23.99607736])}
Calculated in 12.714 seconds


As we are generating a circular waveform only, we could use a faster circular approximant such as `IMRPhenomXPHM`.

In [9]:
params = {'ecc10': 0.4, 'chirp_mass': 22.16087952281798, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0.5}
start = time.time()
zero_ecc_params = find_zero_ecc_chirp(params, 4096, 10, psd, approximant='IMRPhenomXPHM')
print(zero_ecc_params)
end = time.time()
print(f'Calculated in {end-start:.3f} seconds')

{'symmetric_mass_ratio': Array([0.22222222]), 'chi_eff': Array([0.5]), 'ecc10': 0, 'chirp_mass': array([24.02204714])}
Calculated in 3.366 seconds


This greatly speeds up the minimization, but gives a slightly different answer due to waveform systematics. I believe therefore we should also use `IMRPhenomXPHM` to generate the actual circular waveform as well for consistency if we choose to use this approximant.

What about using the peak finding methods from simple-pe to find the best circular chirp mass?

In [82]:
from simple_pe.param_est import find_peak_snr
from simple_pe.param_est.filter import _neg_net_snr
start = time.time()
params = {'ecc10': 0.7, 'chirp_mass': 22.16087952281798, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0.5}
init_guess = {'ecc10': 0, 'chirp_mass': 22.16087952281798, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0.5}
bounds = [(22, 29)]
dx_directions = ['chirp_mass']
target_snr = 20
psd = {'H1': aLIGOZeroDetHighPower((32*4096)//2 + 1, 1/32, 10)}
data = make_waveform(params, psd['H1'].delta_f, 10, len(psd['H1']), 'TEOBResumS-Dali').to_timeseries()
raw_snr = sigma(data, psd['H1'], low_frequency_cutoff=10)
data = {'H1': data.cyclic_time_shift(-1)*target_snr/raw_snr}
peak_dict, snr = find_peak_snr(['H1'], data, psd, data['H1'].sample_times[0], data['H1'].sample_times[-1], init_guess, dx_directions, 10,
                               bounds=bounds, approximant="TEOBResumS-Dali", method='scipy', scipy_method='Nelder-Mead')
end = time.time()
print(f'Calculated in {end-start:.2f} seconds')
print(peak_dict, snr)

Calculated in 49.81 seconds
{'chirp_mass': 24.559296977033114, 'ecc10': 0.0, 'symmetric_mass_ratio': 0.2222222222222222, 'chi_eff': 0.5} 7.562448941513474


In [84]:
fix_params = {'ecc10': 0, 'chirp_mass': 26.26, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0.5}
vary_params = [fix_params.pop(key) for key in dx_directions]
snr = -_neg_net_snr(vary_params, dx_directions, ['H1'], data, psd, data['H1'].sample_times[0], data['H1'].sample_times[-1], 10, approximant="TEOBResumS-Dali", fixed_pars=fix_params)
print(snr)

6.817931552504776
